In [1]:
import multiprocessing as mp
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import os
import math
from tqdm import tqdm
pd.set_option('display.max_rows', 500)
pd.set_option('display.max_columns', 500)
import datetime
from sklearn.preprocessing import quantile_transform
pd.options.mode.chained_assignment = None  # default='warn'
import gc
import requests
from urllib.parse import urlparse

In [2]:
# input data: 주소, 건물 정보. 토지 정보는 자동으로 가져와야 함. 토지 정보를 자동으로 가져오려면 오리지널 토지특성정보 &
# 토지이용계획정보, 좌표는 api에서 다운로드해야함.

In [3]:
# 1. 오리지널 토지특성정보, 토지이용계획정보 & 공시지가
# 2. 좌표
# 3. 건물 정보

In [4]:
# 1. 오리지널 토지특성정보 & 토지이용계획정보

In [5]:
p_land_specs_df = pd.read_csv('./토지특성정보/land_specs_production_ver_1.csv')
print(p_land_specs_df.shape)
p_land_specs_df.head()

(983625, 9)


,지목명,토지면적,용도지역명1,용도지역명2,토지이동상황,지형높이,지형형상,도로접면,지번주소
0,전,876.0,자연녹지지역,지정되지않음,전,완경사,부정형,맹지,서울특별시 강남구 개포동 100-0
1,전,641.0,자연녹지지역,지정되지않음,전,평지,부정형,맹지,서울특별시 강남구 개포동 101-0
2,전,509.0,자연녹지지역,지정되지않음,전,평지,부정형,맹지,서울특별시 강남구 개포동 102-0
3,전,48.7,자연녹지지역,개발제한구역,전,평지,부정형,맹지,서울특별시 강남구 개포동 103-0
4,전,2995.0,개발제한구역,자연녹지지역,전,완경사,부정형,맹지,서울특별시 강남구 개포동 104-1


In [6]:
p_land_plans_df = pd.read_csv('./토지이용계획정보/seoul_land_plans_production_ver_1.csv').fillna('해당사항없음')
print(p_land_plans_df.shape)
p_land_plans_df.head()

c:\users\max6296\ml\lib\site-packages\IPython\core\interactiveshell.py:3147: DtypeWarning: Columns (2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,40,41,42,43,44,45,46,47,48,49,51,52,53,54,55,56,57,58,59,60,61,62,63,64,65,66,67,68,69,70,71,72,73,74,75,76,77,78,79,80,81,82,83,84,85,86,87,88,89,90,91,92,93,94,95,96,97,98,99,100,101,102,103,104,105,106,107,108,109,110,111,112,113,114,115,116,117,118,119,120,121,122,123,124,125,126,127,128,129,130,131,132,133,134,135,136,137,138,139,140,141,142,143,144,145,146,147,148,149,150,151,152,153,154,155,156,157,158,159,160,161,162,163,164,165,166,167,168,169,170,171,172,173,174,175,176,177,178,179,180,181,182,183,184,185,186,187,188,189,190,191,192,193,194,195,196,197,198,199,200,201,202,203,204,205,206,207,208,209,210,211,212,213,214,215,216,217,218,219,220,221,222,223,224,225,226,227,228,229,230,231,232,233,234,235,236,237,238,239,240,241,242,243,244,245,246,247,248,249,250,251,252,253,2

(983564, 350)


,지번주소,년,(한강)오염행위 제한지역,(한강)폐기물매립시설 설치제한지역,4대문안,가로구역별 최고높이 제한지역,가스공급설비,가스공급시설,가축사육제한구역,개발제한구역,개발진흥지구,개발행위허가제한지역,건축선,건축용도지역기타,건축용도지역미분류,건축허가·착공제한지역,경관광장,경관녹지,경관지구,경관지구기타,고가도로,고도지구,고등학교,고속철도,고압선,공간시설미분류,공공공지,공공도서관,공공문화체육시설미분류,공공시설구역,공공시설용지,공공주택지구,공공지원민간임대주택 공급촉진지구,공공청사,공동구,공설화장시설,공용시설보호지구,공원,공원마을지구(공원집단시설지구),공원문화유산지구,공원자연보존지구,공원자연환경지구,공익용산지,공익임지,공장설립승인지역,공장설립제한지역,공항,공항소음피해예상지역,공항소음피해지역,공항시설보호지구,과밀억제권역,광로1류(폭 70M 이상),광로2류(폭 50M~70M),광로3류(폭 40M~50M),광역계획구역,광역복합환승센터,광장,교육환경보호구역,교차점광장,교통광장,교통운수시설미분류,국가산업단지,국가지정문화재구역,국가하천,국립공원,국민임대주택단지예정지구,국지도로,국토이용기타용도지구,국토이용용도지구기타,군사기지 및 군사시설 보호구역,군사시설 보호구역,궤도,근린공원,근린광장,근린상업지역,기업형임대주택 공급촉진지구,기타공공공지시설,기타공공청사시설,기타공원시설,기타교통시설,기타녹지시설,기타도로시설,기타도시공간시설,기타도시방재시설,기타문화시설,기타방송통신시설,기타보건위생시설,기타사회복지시설,기타수도시설,기타시장시설,기타열공급설비,기타용도지역지구기타,기타용도지역지구미분류,기타용지,기타유통및공급시설,기타유통업무설비,기타자동차정류장,기타전기공급설비,기타주차장시설,기타철도시설,기타폐기물처리시설,기타하수도시설,기타학교시설,기타환경기초시설,노외주차장,녹지,농수산물공판장및농수산물종합유통센터,대공방어협조구역,대공방어협조구역(위탁고도:54-236m),대공방어협조구역(위탁고도:77-257m),대로1류(폭 35M~40M),대로2류(폭 30M~35M),대로3류(폭 25M~30M),대학,도로,도로구역,도서관,도시개발구역,도시개발구역기타,도시고속도로,도시관리계획 입안중,도시기타용도지역지구기타,도시기타용도지역지구미분류,도시기타용도지역지구용도지역지구,도시자연공원,도시자연공원구역,도시지역,도시지역기타,도시철도,등록문화재구역,묘지공원,문화공원,문화시설,문화재,문화재보존영향 검토대상구역,문화재보호구역,문화재보호구역기타,문화지구,미관지구기타,박물관,방송통신시설,방수설비,방재지구,방화지구,배수시설,변전소(전원개발사업구역),변전시설,보건위생시설미분류,보전녹지지역,보전산지,보전임지,보조간선도로,보존지구,보행자전용도로,부설주차장,비오톱1등급,비행안전제1구역(전술),비행안전제2구역(전술),비행안전제2구역(지원),비행안전제3구역(전술),비행안전제3구역(지원),비행안전제4구역(전술),비행안전제4구역(지원),비행안전제5구역(전술),비행안전제5구역(지원),비행안전제6구역(전술),사고지,사방시설,사방지,사업지역기타,사업지역미분류,사회복지시설,산업개발진흥지구,산업기술단지,산업시설구역,상대보호구역,상대정화구역,상수원보호구역,상수원보호기타,생산녹지지역,생태·경관보전지역,생태·경관완충보전구역,생태·경관핵심보전구역,소공원,소로1류(폭 10M~12M),소로2류(폭 8M~10M),소로3류(폭 8M 미만),소하천,소하천구역,소하천예정지,수도공급시설,수질오염방지시설,수평표면구역,시·도 생태·경관보전지역,시·도야생생물보호구역,시가지경관지구,시가지조성사업지역,시도지정문화재구역,시장,시장정비구역,시장정비구역기타,시험림구역,아파트지구,액화석유가스충전시설,야생생물보호구역,어린이공원,여객자동차터미널,역사도심,역사문화미관지구,역사문화특화경관지구,역사문화환경보존지역,연결녹지,연구개발특구,연구시설,열공급설비,온천공보호구역,온천원보호지구,완충녹지,용도구역기타,용도구역미분류,용도지구취락지구,운동장,원추표면구역,원형보존지,유류저장및송유설비,유수시설,유수지,유원지,유치원,유통단지,유통상업지역,유통업무설비,일단의공업용지조성사업지역,일단의주택단지조성사업지역,일반광장,일반도로,일반미관지구,일반산업단지,일반상업지역,일반주거지역,일반철도,임업용산지,자동차검사시설,자동차운전학원,자동차전용도로,자동차정류장,자연경관지구,자연공원용도지구기타,자연녹지지역,자연재해위험지구,장애물제한표면구역,재개발구역,재개발구역기타,재정비촉진지구,재정비촉진지구기타,재해위험지구기타,재활용시설,저류시설,전기공급설비,전용주거지역,전이표면구역,전통사찰보존구역,절대보호구역,절대정화구역,정비구역,정비구역기타,정비예정구역,제1종일반주거지역,제1종전용주거지역,제1종지구단위계획구역,제2종일반주거지역,제2종전용주거지역,제3종 구역,제3종일반주거지역,제방,제한보호구역,제한보호구역(방공기지 : 1km),제한보호구역(전술항공:5km),제한보호구역(후방지역:500m),조례로정한지역,조망가로미관지구,조망가로특화경관지구,종합운동장,종합의료시설,주간선도로,주거용지,주거환경개선지구,주요시설광장,주차장,주차장기타,주차환경개선지구,준공업지역,준보전산지,준주거지역,중로1류(폭 20M~25M),중로2류(폭 15M~20M),중로3류(폭 12M~15M),중심대광장,중심상업지역,중심지미관지구,중요시설물보존지구,중요시설물보호지구,중요시설물보호지구(공용),중요시설물보호지구(공항),중점경관관리구역,중학교,지구단위계획구역,지역특화발전특구,지원시설구역,지정문화재구역,지하광장,지하도로,진입표면구역,집단취락지구,집산도로,철도,청사,청소년수련시설,체육공원,체육시설,초등학교,최고고도지구,취수시설,침수위험지구,택지개발예정지구,택지개발예정지구기타,토지거래계약에관한허가구역,토지구획정리사업지구기타,토지형질변경규제지역,통제보호구역,통제보호구역(민통선이남:300m),통제보호구역(방공기지:500m),특수도로(보행자전용도로),특정개발진흥지구,특화경관지구,폐기물처리및재활용시설,폐기물처리시설,하수도,하수종말처리시설,하천,하천구역,하천미분류,학교,학교시설보호지구,학교이적지,학교환경위생 정화구역,현상변경허가 대상구역,홍수관리구역,화장장,환경정비구역
0,서울특별시 강남구 개포동 100-0,2020,해당사항없음,해당사항없음,해당사항없음,해당사항없음,해당사항없음,해당사항없음,해당사항없음,해당사항없음,해당사항없음,해당사항없음,해당사항없음,해당사항없음,해당사항없음,해당사항없음,해당사항없음,해당사항없음,해당사항없음,해당사항없음,해당사항없음,해당사항없음,해당사항없음,해당사항없음,해당사항없음,해당사항없음,해당사항없음,해당사항없음,해당사항없음,해당사항없음,해당사항없음,해당사항없음,해당사항없음,해당사항없음,해당사항없음,해당사항없음,해당사항없음,포함,해당사항없음,해당사항없음,해당사항없음,해당사항없음,접함,해당사항없음,해당사항없음,해당사항없음,해당사항없음,해당사항없음,해당사항없음,해당사항없음,포함,해당사항없음,해당사항없음,해당사항없음,해당사항없음,해당사항없음,해당사항없음,해당사항없음,해당사항없음,해당사항없음,해당사항없음,해당사항없음,해당사항없음,해당사항없음,해당사항없음,해당사항없음,해당사항없음,해당사항없음,해당사항없음,해당사항없음,해당사항없음,해당사항없음,포함,해당사항없음,해당사항없음,해당사항없음,해당사항없음,해당사항없음,해당사항없음,해당사항없음,해당사항없음,해당사항없음,해당사항없음,해당사항없음,해당사항없음,해당사항없음,해당사항없음,해당사항없음,해당사항없음,해당사항없음,해당사항없음,해당사항없음,해당사항없음,해당사항없음,해당사항없음,해당사항없음,해당사항없음,해당사항없음,해당사항없음,해당사항없음,해당사항없음,해당사항없음,해당사항없음,해당사항없음,해당사항없음,해당사항없음,해당사항없음,포함,해당사항없음,해당사항없음,해당사항없음,해당사항없음,해당사항없음,해당사항없음,해당사항없음,해당사항없음,해당사항없음,해당사항없음,해당사항없음,해당사항없음,해당사항없음,해당사항없음,해당사항없음,해당사항없음,해당사항없음,접함,포함,해당사항없음,해당사항없음,해당사항없음,해당사항없음,해당사항없음,해당사항없음,해당사항없음,해당사항없음,해당사항없음,해당사항없음,해당사항없음,해당사항없음,해당사항없음,해당사항없음,해당사항없음,해당사항없음,해당사항없음,해당사항없음,해당사항없음,해당사항없음,해당사항없음,해당사항없음,해당사항없음,접함,해당사항없음,해당사항없음,해당사항없음,해당사항없음,해당사항없음,해당사항없음,해당

In [7]:
p_land_prices_df = pd.read_csv('./공시지가/land_prices_production_ver.csv')
print(p_land_prices_df.shape)
p_land_prices_df.head()

(903493, 5)


,지번주소,특수지구분명,기준년도,공시지가(만원),표준지여부
0,서울특별시 강남구 개포동 100-0,일반,2020,39.60,N
1,서울특별시 강남구 개포동 101-0,일반,2020,47.52,N
2,서울특별시 강남구 개포동 102-0,일반,2020,47.52,N
3,서울특별시 강남구 개포동 103-0,일반,2020,30.97,N
4,서울특별시 강남구 개포동 104-1,일반,2020,39.46,N


In [8]:
# 2. 좌표

In [9]:
# 지번주소로 도로명주소와 좌표를 다운로드함. 출처 - 행정안전부 도로명주소 안내시스템 개발자센터

def fetch_coor_data(addr_list):
    roadname_url = 'https://www.juso.go.kr/addrlink/addrLinkApi.do?'
    roadname_apikey = 'devU01TX0FVVEgyMDIxMDEyNDExMDM0MDExMDczNjE='
    coor_base_url = 'https://www.juso.go.kr/addrlink/addrCoordApi.do?'
    coor_apikey = 'devU01TX0FVVEgyMDIxMDIyNjExMDQwMDExMDg1ODU='
    # 도로명주소 검색 api key 만료일 2021년 4월 24일
    # 좌표검색 api key 만료일 2021년 5월 27일
    #apikey 만료시 https://www.juso.go.kr/addrlink/devAddrLinkRequestWrite.do?returnFn=write&cntcMenu=URL 가서 새로 받을 것.
    
    coor_data_df = pd.DataFrame({'지번주소':[], '도로명주소':[], 'x좌표':[], 'y좌표':[]})
    
    for i, address in tqdm(enumerate(addr_list)):        
        address_no_zero = address.replace('-0','')
        
        roadname_result = requests.get(roadname_url + 'confmKey=' + roadname_apikey + '&currentPage=1' + '&countPerPage=1'\
                                       + '&keyword=' + address_no_zero + '&resultType=json' + '&hstryYn=Y')
        roadname_info = roadname_result.json()['results']['juso']
        
        # 지번주소에 매칭하는 도로명주소가 없을 경우, 근처 지번주소의 도로명주소로 대체
        
        if roadname_info == []:
            splitted = address_no_zero.split('-')
            space_splitted = address_no_zero.split(' ')
            
            if len(splitted) == 1:
                bonbun = int(space_splitted[3])
                bonbun_positive = bonbun
                bonbun_negative = bonbun
                
                while roadname_info == []:
                
                    bonbun_positive += 1
                    bonbun_negative -= 1
                    
                    addr = space_splitted[0] + ' ' + space_splitted[1] + ' ' + space_splitted[2] + ' '\
                    + str(bonbun_positive)
                    
                    roadname_result = requests.get(roadname_url + 'confmKey=' + roadname_apikey + '&currentPage=1'\
                                                   + '&countPerPage=1' + '&keyword=' + addr + '&resultType=json'\
                                                   + '&hstryYn=Y')
                    roadname_info = roadname_result.json()['results']['juso']
                    
                    if roadname_info == []:
                        
                        addr = space_splitted[0] + ' ' + space_splitted[1] + ' ' + space_splitted[2] + ' '\
                        + str(bonbun_negative)
                        
                        roadname_result = requests.get(roadname_url + 'confmKey=' + roadname_apikey + '&currentPage=1'\
                                                       + '&countPerPage=1' + '&keyword=' + addr + '&resultType=json'\
                                                       + '&hstryYn=Y')
                        roadname_info = roadname_result.json()['results']['juso']
                
                
            else:
                bubun = int(splitted[1])
                bubun_positive = bubun
                bubun_negative = bubun
                
                count = 0
                while roadname_info == []:
                    
                    bubun_positive += 1
                    bubun_negative -= 1
                    
                    addr = splitted[0] + '-' + str(bubun_positive)
                    
                    roadname_result = requests.get(roadname_url + 'confmKey=' + roadname_apikey + '&currentPage=1'\
                                                   + '&countPerPage=1' + '&keyword=' + addr + '&resultType=json'\
                                                   + '&hstryYn=Y')
                    roadname_info = roadname_result.json()['results']['juso']
                    
                    if roadname_info == []:
                        
                        addr = splitted[0] + '-' + str(bubun_negative)
                        
                        roadname_result = requests.get(roadname_url + 'confmKey=' + roadname_apikey + '&currentPage=1'\
                                                       + '&countPerPage=1' + '&keyword=' + addr + '&resultType=json'\
                                                       + '&hstryYn=Y')
                        roadname_info = roadname_result.json()['results']['juso']
                    
                    count += 1
                    
                    if count > 10:
                        
                        addr = splitted[0]
                        
                        space_splitted = addr.split(' ')
                        
                        bonbun = int(space_splitted[3])
                        bonbun_positive = bonbun
                        bonbun_negative = bonbun
                        
                        while roadname_info == []:

                            bonbun_positive += 1
                            bonbun_negative -= 1

                            addr = space_splitted[0] + ' ' + space_splitted[1] + ' ' + space_splitted[2] + ' '\
                            + str(bonbun_positive)
                            
                            roadname_result = requests.get(roadname_url + 'confmKey=' + roadname_apikey + '&currentPage=1'\
                                                           + '&countPerPage=1' + '&keyword=' + addr + '&resultType=json'\
                                                           + '&hstryYn=Y')
                            roadname_info = roadname_result.json()['results']['juso']

                            if roadname_info == []:

                                addr = space_splitted[0] + ' ' + space_splitted[1] + ' ' + space_splitted[2] + ' '\
                                + str(bonbun_negative)
                                
                                roadname_result = requests.get(roadname_url + 'confmKey=' + roadname_apikey\
                                                               + '&currentPage=1' + '&countPerPage=1' + '&keyword='\
                                                               + addr + '&resultType=json' + '&hstryYn=Y')
                                roadname_info = roadname_result.json()['results']['juso']
                                
        roadname_info = roadname_info[0]                                
        
        coor_result = requests.get(coor_base_url + 'confmKey=' + coor_apikey + '&admCd=' + roadname_info['admCd']\
                                   + '&rnMgtSn=' + roadname_info['rnMgtSn'] + '&udrtYn=' + roadname_info['udrtYn']\
                                   + '&buldMnnm=' + roadname_info['buldMnnm'] + '&buldSlno=' + roadname_info['buldSlno']\
                                   + '&resultType=json')
        
        coor_info = coor_result.json()['results']['juso'][0]
        
        coor_x = pd.to_numeric(coor_info['entX'])
        coor_y = pd.to_numeric(coor_info['entY'])
        
        roadname_addr = roadname_info['roadAddrPart1']
        
        coor_data_df.loc[i,'지번주소'] = address
        coor_data_df.loc[i,'도로명주소'] = roadname_addr
        coor_data_df.loc[i, 'x좌표'] = coor_x
        coor_data_df.loc[i, 'y좌표'] = coor_y
        
    
    
    return coor_data_df

In [10]:
p_land_specs_df.columns = [col.replace(':','').replace('/','').replace('\\','').replace('*','').replace('?','').replace('\"','')\
                     .replace('|','').replace('<','').replace('>','').replace('ㆍ','_').replace(' ','_') for col in p_land_specs_df.columns]

In [11]:
p_land_specs_df.columns

Index(['지목명', '토지면적', '용도지역명1', '용도지역명2', '토지이동상황', '지형높이', '지형형상', '도로접면',
       '지번주소'],
      dtype='object')

In [12]:
p_land_plans_df.columns = [col.replace(':','').replace('/','').replace('\\','').replace('*','').replace('?','').replace('\"','')\
                     .replace('|','').replace('<','').replace('>','').replace('ㆍ','_').replace(' ','_') for col in p_land_plans_df.columns]

In [13]:
p_land_plans_df.columns

Index(['지번주소', '년', '(한강)오염행위_제한지역', '(한강)폐기물매립시설_설치제한지역', '4대문안',
       '가로구역별_최고높이_제한지역', '가스공급설비', '가스공급시설', '가축사육제한구역', '개발제한구역',
       ...
       '하천구역', '하천미분류', '학교', '학교시설보호지구', '학교이적지', '학교환경위생_정화구역',
       '현상변경허가_대상구역', '홍수관리구역', '화장장', '환경정비구역'],
      dtype='object', length=350)

In [14]:
basedir = './Training/training_data_ver_18/'

In [15]:
training_data_chunk = next(pd.read_csv(basedir + 'training_data_ver_18.csv', chunksize=1))

In [16]:
print(training_data_chunk.shape)
training_data_chunk.head()

(1, 252)


,지번주소,건물연식,금액(만원),가격대지권면적,year_linear,month_sin,month_cos,xnorm,ynorm,x_2nd,y_2nd,xy,공시지가(만원),표준지여부,지목명,용도지역명1,용도지역명2,토지이동상황,지형높이,지형형상,도로접면,(한강)폐기물매립시설_설치제한지역,4대문안,가로구역별_최고높이_제한지역,가스공급설비,가축사육제한구역,개발제한구역,개발행위허가제한지역,건축선,건축용도지역기타,건축용도지역미분류,건축허가·착공제한지역,경관녹지,경관지구,경관지구기타,고도지구,고등학교,고속철도,공공공지,공공시설구역,공공주택지구,공공지원민간임대주택_공급촉진지구,공공청사,공원,공원자연보존지구,공원자연환경지구,공익용산지,공익임지,공장설립승인지역,공장설립제한지역,공항소음피해예상지역,공항시설보호지구,과밀억제권역,광로1류(폭_70M_이상),광로2류(폭_50M~70M),광로3류(폭_40M~50M),광장,교육환경보호구역,교통광장,국가산업단지,국가지정문화재구역,국가하천,국립공원,국민임대주택단지예정지구,국지도로,국토이용용도지구기타,근린공원,근린광장,근린상업지역,기타공공공지시설,기타공공청사시설,기타공원시설,기타교통시설,기타녹지시설,기타도로시설,기타도시공간시설,기타보건위생시설,기타수도시설,기타자동차정류장,기타주차장시설,기타폐기물처리시설,노외주차장,녹지,대공방어협조구역,대공방어협조구역(위탁고도54-236m),대공방어협조구역(위탁고도77-257m),대로1류(폭_35M~40M),대로2류(폭_30M~35M),대로3류(폭_25M~30M),대학,도로,도로구역,도서관,도시개발구역,도시관리계획_입안중,도시기타용도지역지구기타,도시기타용도지역지구미분류,도시기타용도지역지구용도지역지구,도시자연공원,도시자연공원구역,도시지역,도시지역기타,도시철도,문화공원,문화시설,문화재,문화재보존영향_검토대상구역,문화재보호구역,문화지구,박물관,방송통신시설,방수설비,방재지구,방화지구,변전소(전원개발사업구역),보전산지,보전임지,보행자전용도로,부설주차장,비오톱1등급,비행안전제2구역(전술),비행안전제2구역(지원),비행안전제3구역(전술),비행안전제3구역(지원),비행안전제4구역(지원),비행안전제5구역(전술),비행안전제5구역(지원),비행안전제6구역(전술),사고지,사방지,사업지역기타,사업지역미분류,사회복지시설,산업개발진흥지구,산업시설구역,상대보호구역,상대정화구역,상수원보호기타,생산녹지지역,소공원,소로1류(폭_10M~12M),소로2류(폭_8M~10M),소로3류(폭_8M_미만),소하천,소하천구역,수도공급시설,수평표면구역,시가지조성사업지역,시도지정문화재구역,시장,시장정비구역,아파트지구,야생생물보호구역,어린이공원,여객자동차터미널,역사도심,역사문화미관지구,역사문화특화경관지구,역사문화환경보존지역,연결녹지,연구개발특구,열공급설비,온천공보호구역,완충녹지,용도구역기타,원추표면구역,원형보존지,유수지,유원지,유통업무설비,일단의공업용지조성사업지역,일단의주택단지조성사업지역,일반미관지구,일반산업단지,일반상업지역,일반주거지역,일반철도,임업용산지,자동차정류장,자연경관지구,자연녹지지역,자연재해위험지구,장애물제한표면구역,재개발구역,재정비촉진지구,재정비촉진지구기타,저류시설,전기공급설비,전용주거지역,전이표면구역,전통사찰보존구역,절대보호구역,절대정화구역,정비구역,정비구역기타,정비예정구역,제1종일반주거지역,제1종전용주거지역,제1종지구단위계획구역,제2종일반주거지역,제2종전용주거지역,제3종_구역,제3종일반주거지역,제한보호구역,제한보호구역(전술항공5km),제한보호구역(후방지역500m),조례로정한지역,조망가로미관지구,조망가로특화경관지구,주거환경개선지구,주차장,주차장기타,주차환경개선지구,준공업지역,준보전산지,준주거지역,중로1류(폭_20M~25M),중로2류(폭_15M~20M),중로3류(폭_12M~15M),중심지미관지구,중요시설물보존지구,중요시설물보호지구,중요시설물보호지구(공항),중점경관관리구역,중학교,지구단위계획구역,지역특화발전특구,지원시설구역,지하도로,진입표면구역,집단취락지구,철도,청소년수련시설,체육공원,체육시설,초등학교,최고고도지구,침수위험지구,택지개발예정지구,토지거래계약에관한허가구역,토지구획정리사업지구기타,통제보호구역(민통선이남300m),특수도로(보행자전용도로),특정개발진흥지구,폐기물처리시설,하수도,하천,하천구역,학교,학교시설보호지구,학교환경위생_정화구역,현상변경허가_대상구역
0,서울특별시 강남구 대치동 894-0,30,300000.0,1857.585139,1.583333,-0.5,-0.866025,6030.985219,-6274.443335,1.818639e+07,1.968432e+07,-3.784108e+07,1121.0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0


In [17]:
cols_to_drop = ['금액(만원)', '가격대지권면적', '지번주소']

In [18]:
training_features = training_data_chunk.columns.drop(cols_to_drop)

In [19]:
import lightgbm as lgb

In [20]:
model_names_list = [f for f in os.listdir(basedir) if 'lightgbm' in f]
model_names_list

['lightgbm_mse_0.txt',
 'lightgbm_mse_1.txt',
 'lightgbm_mse_2.txt',
 'lightgbm_mse_3.txt',
 'lightgbm_mse_4.txt']

In [21]:
models_list = []
for f in tqdm(model_names_list):
    model = lgb.Booster(model_file=basedir + f)
    models_list.append(model)

100%|████████████████████████████████████████████████████████████████████████████████████| 5/5 [00:00<00:00,  5.41it/s]


In [22]:
def month_circular_sine_func(x):
    return math.sin(2*math.pi*((x-1)/12))

def month_circular_cosine_func(x):
    return math.cos(2*math.pi*((x-1)/12))

In [72]:
def date_modifier(x):
    # x is a string or datetime
    if pd.isna(x) == True:
        return x
    else:
        x = str(x)
        if '-' in x:
            x = x.replace('-', '.')
            
        splitted = x.split('.')
        
        if len(splitted) == 1:
            return np.nan
        else:
            return x

In [65]:
def datetime_feature_engi(df):
    df = df.copy()
    
    #current_date = datetime.datetime.now()
    
    year = df['기준시점'].apply(lambda x: int(str(x).split('.')[0]))
    month = df['기준시점'].apply(lambda x: int(str(x).split('.')[1]))
    
    df['year_linear'] = (year - 2006) + ((month-1)/12)
    
    df['month_sin'] = month_circular_sine_func(month)
    df['month_cos'] = month_circular_cosine_func(month)
    
    return df

In [24]:
def coor_feature_engi(coor_df, coor_minmax_df):
    coor_df = coor_df.copy()
    coor_minmax_df = coor_minmax_df.copy()
    
    coor_df['xnorm'] = (coor_df['x좌표'] - coor_minmax_df['xmin'] - ((coor_minmax_df['xmax'] - coor_minmax_df['xmin'])/2))
    coor_df['ynorm'] = (coor_df['y좌표'] - coor_minmax_df['ymin'] - ((coor_minmax_df['ymax'] - coor_minmax_df['ymin'])/2))
    coor_df['x_2nd'] = coor_df['xnorm']**2 / 2
    coor_df['y_2nd'] = coor_df['ynorm']**2 / 2
    coor_df['xy'] = coor_df['xnorm'] * coor_df['ynorm']
    
    coor_df = coor_df.drop(columns=['x좌표', 'y좌표'])
    
    return coor_df

In [35]:
def prediction_pipeline(models_list, manual_input_df, land_specs_df, land_plans_df, land_prices_df, selected_features_list, verbose=1):
    manual_input_df = manual_input_df.copy()
    land_specs_df = land_specs_df.copy()
    land_plans_df = land_plans_df.copy()
    land_prices_df = land_prices_df.copy()
    
    prediction_data_df = manual_input_df.merge(land_specs_df, on=['지번주소'], how='left')\
    .merge(land_prices_df, on=['지번주소'], how='left').merge(land_plans_df, on=['지번주소'], how='left')
    
    #prediction_data_df['건물나이'] = prediction_data_df['건물나이'].apply(building_age_feature_engi)
    
    addr_list = manual_input_df['지번주소'].unique().tolist()
    
    coor_df = fetch_coor_data(addr_list)
    coor_df = coor_df[['지번주소', 'x좌표', 'y좌표']]
    
    coor_minmax_df = pd.read_csv('./Training/training_data_ver_18/coor_minmax.csv')
    
    coor_df = coor_feature_engi(coor_df, coor_minmax_df)
    
    prediction_data_df = prediction_data_df.merge(coor_df, on=['지번주소'], how='left')
    
    #prediction_data_df['건축년도'] = 2020
    
    prediction_data_df = datetime_feature_engi(prediction_data_df)
    
    '''x_cols = training_data_df.columns.drop(['주택가격지수', '가격면적', 'targetarea_log_transformed', '금액(만원)', 
                                            'target_log_transformed'
                                           ]).tolist()'''
    
    '''prediction_data_df['토지면적over총연면적times공시지가'] = (prediction_data_df['토지면적'] / prediction_data_df['총연면적'])\
    * prediction_data_df['공시지가(만원)']'''
    
    prediction_data_df = prediction_data_df[selected_features_list]
    
    cols_to_log = ['총건축면적', '총연면적', '총용적률연면적', '토지면적', '부속건축물면적']
    for col in cols_to_log:
        if col in prediction_data_df.columns.tolist():
            prediction_data_df[col] = prediction_data_df[col].apply(lambda x: np.log1p(float(x)))
        
    cat_cols_df = pd.read_csv('./Training/training_data_ver_18/cat_cols.csv')
    
    cat_cols = []
    for col in cat_cols_df['colname']:
        if col in prediction_data_df.columns:
            cat_cols.append(col)
    
    if verbose == 1:
        print('mapping categorical data...')
    
        for col in tqdm(cat_cols):
            col_map_df = pd.read_csv('./Training/training_data_ver_18/feature_maps/{}_map.csv'.format(col))
            col_map = {}
            for i in range(col_map_df.shape[0]):
                col_map[col_map_df['cat'][i]] = col_map_df['map'][i]
            prediction_data_df[col] = prediction_data_df[col].map(col_map)
    
    elif verbose == 0:
        for col in cat_cols:
            col_map_df = pd.read_csv('./Training/training_data_ver_18/feature_maps/{}_map.csv'.format(col))
            col_map = {}
            for i in range(col_map_df.shape[0]):
                col_map[col_map_df['cat'][i]] = col_map_df['map'][i]
            prediction_data_df[col] = prediction_data_df[col].map(col_map)
        
    
    return prediction_data_df

In [93]:
reference_df = pd.read_excel('./Production_Prep/이승준_총괄표.xlsx', header=1)
print(reference_df.shape)
reference_df.head()

(137, 39)


,지역,소재지,지번,용도지역,도로너비,토지면적(㎡),토지매매금액,토지매매단가\n(원/㎡),공시지가와격차\n(개공/매매),건물용도,공급면적(㎡),전유면적(㎡),전용율,용적율\n(%),건폐율\n(%),구분건물호,건물동수,건물층수,의뢰인,기준시점,전체평가금액,투입비용합계,순이익,순이익율,전유면적,공급면적,전유면적기준,공급면적기준,토지계약일자,허가일자,착공일자,사용승인일,Unnamed: 32,토지원가비율,건축원가비율,부대비용,금융비용,합계,Unnamed: 38
0,강동구,천호동,17-4외,2종일주,4~6미터,934.0,5.549000e+09,5941113.0,0.541,도시형생활주택,1720.57,1418.72,0.825,184.22,55.98,27개호,3개동,지상5층,SH공사,2019.12.18,1.005050e+10,9.128694e+09,921806160.0,0.092,7084203.0,5.841088e+06,6.434458e+06,5515021.0,2019.04.04,2019.11.26,2019.12.26,2020.07.09,5500000.0,0.6473,0.3233,0.0091,0.0203,1.0,NaN
1,강동구,천호동,18-65외,2종일반주거,6미터,261.0,1.880000e+09,7203065.0,0.484,도시형생활주택,517.56,414.78,0.689,199.83,53.57,15개호,1개동,지하1\n지상6,LH공사,2019.06.07,3.633000e+09,3.109518e+09,523482390.0,0.144,8758860.0,6.034783e+06,7.496788e+06,5932890.0,2018.06.13,2018.09.18,2018.10.19,2019.04.11,5400000.0,0.6423,0.3262,0.0091,0.0224,1.0,NaN
2,강동구,천호동,18-163외,2종일반주거,4미터,228.0,1.510000e+09,6622807.0,0.490,도시형생활주택,454.94,378.30,0.832,199.54,58.35,14세대,1개동,14세대,SH공사,2020.11.23,3.091500e+09,2.448262e+09,643238380.0,0.208,8172086.0,6.795402e+06,6.471746e+06,5588504.0,2019.09.18,2020.04.24,2020.05.21,2020.11.11,5400000.0,0.6552,0.3131,0.0088,0.0229,1.0,NaN
3,강동구,천호동,36-8외,2종일주,4미터미만,267.4,1.860000e+09,6955871.0,0.530,다세대주택,534.75,467.37,0.874,199.54,58.35,10세대,1개동,10세대,SH공사,2020.10.23,3.232500e+09,2.992845e+09,239655010.0,0.074,6916362.0,6.044881e+06,6.403588e+06,5827722.0,2019.10.30,2020.02.10,2020.03.30,2020.10.08,5500000.0,0.6612,0.3065,0.0086,0.0237,1.0,NaN
4,강동구,암사동,433-69외,3종일주,4미터,258.0,1.643000e+09,6368217.0,0.479,다세대주택,493.67,408.01,0.850,209.57,58.46,9세대,1개동,지상5층,SH공사,2020.10.23,2.901000e+09,2.657727e+09,243272910.0,0.084,7110120.0,6.040855e+06,6.513877e+06,5319617.0,2019.11.11,2020.02.14,2020.05.04,2020.09.28,5500000.0,0.6573,0.3100,0.0087,0.0241,1.0,NaN


In [94]:
reference_df['토지면적단가'] = reference_df['전체평가금액'] / reference_df['토지면적(㎡)']

In [95]:
reference_df['토지면적단가'] = reference_df['토지면적단가'] / 10000

In [96]:
def jibun_modifier(x):
    # x is a string
    
    if pd.isna(x) == True:
        return x
    else:
        x = str(x)
        x = x.replace('외', '')
        splitted = x.split('-')
        if len(splitted) == 1:
            return x + '-0'
        else:
            return x

In [97]:
reference_df['지번주소'] = '서울특별시 ' + reference_df['지역'] + ' ' + reference_df['소재지'] + ' ' + reference_df['지번'].apply(jibun_modifier)

In [98]:
reference_df['예측토지면적단가'] = np.nan

In [99]:
reference_df['기준시점'] = reference_df['기준시점'].apply(date_modifier)

In [100]:
reference_df['기준시점'].unique()

array(['2019.12.18', '2019.06.07', '2020.11.23', '2020.10.23',
       '2019.04.01', '2019.06.14', '2020.05.25', '2020.11.11',
       '2020.07.20', '2020.09.23', '2019.11.29', '2019.12.27',
       '2020.10.13', '2020.03.10', '2020.01.10', '2020.10.06',
       '2020.10.30', '2020.07.23', '2020.06.25', '2020.09.04',
       '2020.12.05', '2020.04.27', '2020.05.27', '2020.12.27',
       '2020.08.27', '2019.11.12', '2019.11.22', '2019.04.22',
       '2020.06.18', '2019.01.28', '2019.09.10', '2019.01.03',
       '2019.01.18', '2019.12.13', '2019.05.27', '2019.05.14',
       '2019.11.22 00:00:00', '2019.03.04 00:00:00',
       '2020.08.13 00:00:00', '2020.07.10 00:00:00',
       '2020.06.08 00:00:00', '2020.06.18 00:00:00',
       '2020.07.17 00:00:00', '2020.08.27 00:00:00',
       '2020.01.28 00:00:00', '2020.09.21 00:00:00',
       '2020.12.10 00:00:00', nan, '2019.06.24 00:00:00',
       '2019.01.28 00:00:00', '2018.03.23 00:00:00',
       '2019.12.06 00:00:00', '2019.06.20 00:00:00',
    

In [101]:
reference_df.dropna(subset=['지번주소', '기준시점'], inplace=True)
reference_df = reference_df.reset_index(drop=True)

In [102]:
reference_df.shape

(86, 42)

In [104]:
for i in tqdm(range(reference_df.shape[0])):
    addr = reference_df.loc[i, '지번주소']
    r_date = reference_df.loc[i, '기준시점']
    
    if (pd.isna(addr) == True):
        continue
    
    manual_input_df = pd.DataFrame({'지번주소':[addr], '건물연식':[0], '기준시점':[r_date]})
    
    prac_df = prediction_pipeline(models_list, manual_input_df, p_land_specs_df, p_land_plans_df, p_land_prices_df, training_features, verbose=0)
    
    if prac_df.isna().sum().sum() > 0:
        continue
    
    preds_list = []
    for model in models_list:
        preds = model.predict(prac_df)
        preds_list.append(preds)
        
    reference_df.loc[i, '예측토지면적단가'] = np.mean(preds_list)

  0%|                                                                                           | 0/86 [00:00<?, ?it/s]
0it [00:00, ?it/s]
1it [00:00,  5.15it/s]
  1%|▉                                                                                  | 1/86 [00:10<15:25, 10.89s/it]
0it [00:00, ?it/s]
1it [00:00,  3.89it/s]
  2%|█▉                                                                                 | 2/86 [00:20<14:26, 10.31s/it]
0it [00:00, ?it/s]
1it [00:00,  4.90it/s]
  3%|██▉                                                                                | 3/86 [00:30<13:48,  9.98s/it]
0it [00:00, ?it/s]
1it [00:00,  4.48it/s]
  5%|███▊                                                                               | 4/86 [00:39<13:24,  9.82s/it]
0it [00:00, ?it/s]
1it [00:00,  4.50it/s]
  6%|████▊                                                                              | 5/86 [00:49<12:55,  9.57s/it]
0it [00:00, ?it/s]
1it [00:00,  4.78it/s]
  7%|█████▊                 

In [107]:
reference_df.dropna(subset=['토지면적단가', '예측토지면적단가'], inplace=True)

In [108]:
from sklearn.metrics import mean_squared_error

In [109]:
np.sqrt(mean_squared_error(reference_df['토지면적단가'], reference_df['예측토지면적단가']))

136.9019679165493

In [110]:
reference_df.to_excel('./Production_Prep/이승준_총괄표_토지단가_예측포함.xlsx')

In [111]:
inv_df = reference_df[['지번주소', '건물용도', '의뢰인', '기준시점', '토지면적단가', '예측토지면적단가']]
inv_df.shape

(84, 6)

In [112]:
inv_df

,지번주소,건물용도,의뢰인,기준시점,토지면적단가,예측토지면적단가
0,서울특별시 강동구 천호동 17-4,도시형생활주택,SH공사,2019.12.18,1076.070664,1086.142558
1,서울특별시 강동구 천호동 18-65,도시형생활주택,LH공사,2019.06.07,1391.954023,1414.304106
2,서울특별시 강동구 천호동 18-163,도시형생활주택,SH공사,2020.11.23,1355.921053,1359.307465
3,서울특별시 강동구 천호동 36-8,다세대주택,SH공사,2020.10.23,1208.863126,1253.515959
4,서울특별시 강동구 암사동 433-69,다세대주택,SH공사,2020.10.23,1124.418605,1155.316914
5,서울특별시 강동구 암사동 454-26,다세대주택,SH공사,2019.04.01,1116.701147,1156.493326
6,서울특별시 강동구 암사동 455-51,다세대주택,LH공사,2019.04.01,1235.948645,1333.669877
7,서울특별시 강동구 암사동 457-30,다세대주택,SH공사,2019.06.14,1072.149644,1089.414475
8,서울특별시 강동구 고덕동 292-5,다세대주택,LH공사,2019.06.14,1460.648148,1419.613689
9,서울특별시 강동구 고덕동 292-6,다세대주택,SH공사,2019.06.14,1463.029067,1498.996671


In [113]:
inv_df['오차'] = inv_df['토지면적단가'] - inv_df['예측토지면적단가']

In [114]:
inv_df['오차율'] = inv_df['오차'] / inv_df['토지면적단가'] * 100

In [115]:
inv_df.rename(columns={'오차율':'오차율(%)'}, inplace=True)

In [116]:
dasede_df = inv_df[inv_df['건물용도'].str.contains('다세')]
print(dasede_df.shape)
dasede_df.head()

(49, 8)


,지번주소,건물용도,의뢰인,기준시점,토지면적단가,예측토지면적단가,오차,오차율(%)
3,서울특별시 강동구 천호동 36-8,다세대주택,SH공사,2020.10.23,1208.863126,1253.515959,-44.652833,-3.693787
4,서울특별시 강동구 암사동 433-69,다세대주택,SH공사,2020.10.23,1124.418605,1155.316914,-30.898309,-2.747937
5,서울특별시 강동구 암사동 454-26,다세대주택,SH공사,2019.04.01,1116.701147,1156.493326,-39.792179,-3.563369
6,서울특별시 강동구 암사동 455-51,다세대주택,LH공사,2019.04.01,1235.948645,1333.669877,-97.721233,-7.906577
7,서울특별시 강동구 암사동 457-30,다세대주택,SH공사,2019.06.14,1072.149644,1089.414475,-17.264832,-1.610301


In [117]:
np.sqrt(mean_squared_error(dasede_df['토지면적단가'], dasede_df['예측토지면적단가']))

113.33165747580712

In [118]:
dasede_df['오차율절대값'] = dasede_df['오차율(%)'].apply(np.abs)

In [119]:
dasede_df = dasede_df.sort_values(['오차율절대값'], ascending=False)

In [120]:
dasede_df

,지번주소,건물용도,의뢰인,기준시점,토지면적단가,예측토지면적단가,오차,오차율(%),오차율절대값
74,서울특별시 금천구 독산동 378-96,다세대,SH공사,2019.06.24 00:00:00,843.697479,1144.639764,-300.942285,-35.669454,35.669454
76,서울특별시 금천구 시흥동 203-4,다세대주택,SH공사,2018.03.23 00:00:00,761.179361,1004.398595,-243.219234,-31.952947,31.952947
51,서울특별시 구로구 가리봉동 103-1,다세대주택,SH공사,2019.01.03,949.462366,1247.387329,-297.924963,-31.378280,31.378280
52,서울특별시 구로구 가리봉동 134-22,다세대주택,SH공사,2019.01.18,888.763091,1108.813188,-220.050098,-24.759140,24.759140
53,서울특별시 구로구 가리봉동 771-3,도생-다세대,LH공사,2019.12.13,930.300910,1137.013395,-206.712485,-22.219960,22.219960
58,서울특별시 구로구 개봉동 109-26,다세대주택,LH공사,2019.11.22 00:00:00,840.000000,1023.393201,-183.393201,-21.832524,21.832524
50,서울특별시 구로구 가리봉동 87-63,다세대주택,LH공사,2019.12.18,1055.858311,1274.647145,-218.788835,-20.721420,20.721420
57,서울특별시 구로구 개봉동 109-25,다세대주택,LH공사,2019.11.22 00:00:00,859.808612,1020.478043,-160.669431,-18.686651,18.686651
54,서울특별시 구로구 개봉동 7-26,도생-다세대,LH공사,2019.11.22,965.123457,1109.817294,-144.693837,-14.992262,14.992262
24,서울특별시 강북구 미아동 164-2,단지형다세대,LH공사,2019.11.29,1042.079208,1190.295770,-148.216563,-14.223157,14.223157


In [121]:
dasede_df['오차율절대값'].mean()

8.511825945275053